In [16]:
#junzhimoxing
import os
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

def mkdir(path):
    isExists=os.path.exists(path)
    if not isExists:
        os.makedirs(path) 


path_to_bj_aq = "./data/months1_10/"

bj_csv_list  = os.listdir(path_to_bj_aq)

for csv in bj_csv_list :
    if csv != '.DS_Store' and not csv.startswith("._"):
        path_to_file = path_to_bj_aq + csv
        
        data = pd.read_csv(path_to_file)
        i = csv.split('.')[0]

        data['time_stamp'] = pd.to_datetime(data['time_stamp'])
        data = data.groupby(['time_stamp','loc_id'],as_index = False).count()
        data['date'] = data['time_stamp'].apply(lambda x:str(x)[:10])
        data['hour'] = data['time_stamp'].apply(lambda x:int(str(x)[11:13]))
        data['week'] = data['time_stamp'].apply(lambda x:x.weekday())

        for j in range(1,34):
            data_part = data[data['loc_id'] == j]
            mkdir('./data/station_%s' % (j))
            data_part.to_csv('data/station_%s/month_%s.csv' % (j,i),index = False)










data = pd.DataFrame()

min_time = datetime.datetime.strptime('2017-12-01 00:00:00', '%Y-%m-%d %H:%M:%S')
max_time = datetime.datetime.strptime('2017-12-31 23:00:00', '%Y-%m-%d %H:%M:%S')

delta_all = max_time - min_time
hours_should = delta_all.total_seconds()/3600 + 1
delta = datetime.timedelta(hours=1)
time = min_time

time_stamp = []
loc_id = []
while time <=  max_time :
    
    for i in range(1,34):
        time_stamp.append(time)
        loc_id.append(i)
    time += delta
data['time_stamp'] = time_stamp
data['loc_id'] = loc_id


data['time_stamp'] = pd.to_datetime(data['time_stamp'])
data['date'] = data['time_stamp'].apply(lambda x:str(x)[:10])
data['hour'] = data['time_stamp'].apply(lambda x:int(str(x)[11:13]))
data['week'] = data['time_stamp'].apply(lambda x:x.weekday())

holiday = pd.read_csv('data/holiday.csv')
data = pd.merge(data, holiday, how='inner', on=['date'])
data['holiday'][data['holiday'] == 2] = 1
# data['data_mean'] = 0
# data['data_max'] = 0
# data['data_min']= 0
# data['data_median'] = 0

# # data['data_mean_hour'] = 0
# # data['data_max_hour'] = 0
# # data['data_min_hour']= 0
# data['data_median_hour'] = 0

# # data['data_mean_week'] = 0
# # data['data_max_week'] = 0
# # data['data_min_week']= 0
# data['data_median_week'] = 0




data['data_mean_week_hour'] = 0
# data['data_max_week_hour'] = 0
# data['data_min_week_hour']= 0
data['data_median_week_hour'] = 0
data['data_var_week_hour'] = 0
datas = []
for i in range(1,34):
    path_to_bj_aq = "./data/station_"+str(i)+"/"

    bj_csv_list  = os.listdir(path_to_bj_aq)
    for csv in bj_csv_list :
        if csv != '.DS_Store' and not csv.startswith("._"):
            path_to_file = path_to_bj_aq + csv
            data_su = pd.read_csv(path_to_file)
            if data_su.empty == False:
                datas.append(data_su)

data_all = pd.concat(datas, ignore_index=True)
data_all.rename(index=str, columns={"phone_id": "phone_num"},inplace = True)

# for i in range(1,34):    
#     data_mean = data_all['phone_num'][data_all['loc_id'] == i].mean()
#     data_max = data_all['phone_num'][data_all['loc_id'] == i].max()
#     data_min = data_all['phone_num'][data_all['loc_id'] == i].min()
#     data_median = data_all['phone_num'][data_all['loc_id'] == i].median()
    
#     data['data_mean'][data['loc_id'] == i] = data_mean
#     data['data_max'][data['loc_id'] == i] = data_max
#     data['data_min'][data['loc_id'] == i] = data_min
#     data['data_median'][data['loc_id'] == i] = data_median
        
for i in range(1,34):
    for h in range(24):   
        for j in range(7):
            data_mean = data_all['phone_num'][(data_all['loc_id'] == i)&(data_all['week'] == j)&(data_all['hour'] == h)].mean()
#             data_max = data_all['phone_num'][(data_all['loc_id'] == i)&(data_all['week'] == j)&(data_all['hour'] == h)].max()
#             data_min = data_all['phone_num'][(data_all['loc_id'] == i)&(data_all['week'] == j)&(data_all['hour'] == h)].min()
            data_median = data_all['phone_num'][(data_all['loc_id'] == i)&(data_all['week'] == j)&(data_all['hour'] == h)].median()
            data_var = data_all['phone_num'][(data_all['loc_id'] == i)&(data_all['week'] == j)&(data_all['hour'] == h)].var()
#             print(data_mean)

            data['data_mean_week_hour'][(data['loc_id'] == i)&(data['week'] == j)&(data['hour'] == h)] = data_mean
#             data['data_max_week_hour'][(data['loc_id'] == i)&(data['week'] == j)&(data['hour'] == h)] = data_max
#             data['data_min_week_hour'][(data['loc_id'] == i)&(data['week'] == j)&(data['hour'] == h)] = data_min
            data['data_median_week_hour'][(data['loc_id'] == i)&(data['week'] == j)&(data['hour'] == h)] = data_median
            data['data_var_week_hour'][(data['loc_id'] == i)&(data['week'] == j)&(data['hour'] == h)] = data_var
# data['time_stamp'] = data['time_stamp'].astype('str')




data.to_csv('predict12_latiao.csv',index = False)

/home/zhangqq/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhangqq/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhangqq/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhangqq/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:141: SettingWithCopyWarning: 
A value is trying 

In [17]:
train = pd.read_csv('./predict12_latiao.csv')
train['time_stamp'] = train['time_stamp'].apply(lambda x: x.split(':')[0])  
train = train[['loc_id','time_stamp','data_mean_week_hour']]
train.to_csv('submitkankan_qushang11.csv',index = False)



In [29]:
train = pd.read_csv('./submitkankan_qushang11.csv')

In [30]:
train['num_of_people'] = train['num_of_people']*0.96
train.to_csv('submitzqq96.csv',index=False)